# People Matching Algortihm

## Steps to be done

1. Prepare and clean the datasets
2. Calculate similarity within the two user groups
3. Assign mentor to each mentee based on similarity
3. Filter the assignments by the following preferences:
    * Mentee's priority nr 1: theatre, role, segment or gender
    * Mentor's prefered theatre
    * Mentor's prefered role
    * Mentor's prefered gender
4. Filter by mentor, if he/she has a mentee in mind
5. Shuffle the assignments until every mentee has a unique mentor assigned (shuffle top 4 mentors)
6. Exporting to excel sheet by ranking


# 1. Prepare and clean the datasets
## 1.1. Mentor Data

In [539]:
import pandas as pd
import scipy.spatial
import random
import xlsxwriter
import warnings
warnings.simplefilter("ignore")

In [540]:
# reed in mentor data
df_mentor_raw = pd.read_excel(r"FY21 - Chairman Mentor Interest Form.xlsx",sheet_name="Form_a")

# check out data
df_mentor_raw.shape  #(91, 41)

(85, 41)

In [541]:
# function that drops a column if it contains only NaN values
def drop_nan_cols(df):
    nr = df.shape[0]
    for column in df.columns:
        if df[column].isnull().sum(axis = 0) == nr:
            df.drop([column], axis=1, inplace=True)
    return df

df_mentor_raw = drop_nan_cols(df_mentor_raw)
df_mentor_raw.shape  #(91, 29) removed 12 cols full of NaNs

(85, 29)

In [542]:
# remove first two entries since these were test entries
# df_mentor_raw.drop([0,1], axis=0, inplace=True)

# replace the numbered index by mentor names
df_mentor_raw = pd.DataFrame(df_mentor_raw).set_index('Name', drop=True)

In [543]:
df_mentor_raw.head()

,ID,Start time,Completion time,How many Y2 CSAP Associates are you willing to mentor?,Gender,Are you a CSAP Alum?2,What is your CSAP Class FY?,Which track were you in?,Have you mentored an Early in Career before?,What are you most likely to give in this mentoring engagement?,...,What was most challenging about the engagement and how did you overcome it?,Your country of residence,What are your general strengths?,What are your personal interests?,What do you think qualifies you to be a good mentor?2,What are your early expectations from this mentoring engagement?,Your linkedIn profile link,Have you already mentored for the Chairman Club Mentoring Program?,Any final comments or suggestions before you submit?,Let us know if you already have a CSAP mentee in mind
Name,,,,,,,,,,,,,,,,,,,,,
22341,57,44175.679884,44175.684433,1,M,No,NaN,NaN,No,I'm flexible and will adapt to my mentee's goals;,...,NaN,Brazil,Design thinking;Cross-architecture;Story telling;,"Team sports (soccer, football);Travel;Music;","Experience, respect, listening, generosity, ho...",Help the mentee accelerate his/her development...,https://www.linkedin.com/in/adriano-alves-9a7298/,No,NaN,NaN
22342,67,44179.689433,44179.700625,2,F,No,NaN,NaN,Yes,I'm flexible and will adapt to my mentee's goals;,...,Skill development through opportunities for le...,USA,Customer interaction;Influencing;Team building;,Cooking;Pets;Travel;,I'm motivated through my contributions to help...,Help provide insight and guidance from a CX pe...,https://www.linkedin.com/in/alyssahaglund/,No,Looking forward to seeing if I can help :),NaN
22343,31,44172.801921,44172.804630,1,F,No,NaN,NaN,No,Sharing my own career journey and lessons lear...,...,NaN,USA,Accountability;Collaboration;Communications (p...,Cooking;Culture;Design;Gardening;Individual sp...,"Candid feedback, honesty and the tools to prov...","Providing guidance, support and tools for the ...",https://www.linkedin.com/in/amaniturminini,No,Thank you for the opportunity.,NaN
22344,87,44182.376516,44183.028275,2,M,Yes,FY09,ASE,No,I'm flexible and will adapt to my mentee's goals;,...,NaN,Australia,Influencing;Customer interaction;Competitive s...,"Family;Team sports (soccer, football);Travel;",I,To both provide the guidance that was afforded...,https://www.linkedin.com/in/andrew-bernath-395...,No,NaN,NaN
22345,46,44173.561331,44173.566944,2,F,No,NaN,NaN,Yes,General strategic career advice depending on m...,...,Time frame \nAnd over came it by being persist...,UAE,Negotiating;Personal Brand;Team building;Criti...,Art;Family;Volunteering;Reading;Music;,Knowledge and skills \nAbility to see the best...,Develop and progress \nExpand my knowledge \nG...,http://linkedin.com/in/ansam-abuljadayel-84809758,No,NaN,NaN


Up to this point, the dataset is clean from data we do not need for matching. 

In [544]:
type(df_mentor_raw)

pandas.core.frame.DataFrame

In [545]:
#df_mentor_raw.isnull().sum(axis = 0)

In [546]:
df_mentor = df_mentor_raw[["What are you most likely to give in this mentoring engagement?",
                           "What are your general strengths?",
                           "What are your personal interests?"]]

#df_mentor.head()

In [547]:
# function that splits the choices in the colums by ; to and adds them to a list
def extract_choices(df_col):
    choices = []
    cols = set()
    data = set(df_col)
    for choice in data:
        choice = choice.split(";")
        choices.append(choice)
    for innerlist in choices:
        for choice in innerlist:
            if choice != "":
                cols.add(choice)    
        
    return list(cols)

give_mentor = extract_choices(df_mentor["What are you most likely to give in this mentoring engagement?"])
strengths_mentor = extract_choices(df_mentor["What are your general strengths?"])
interests_mentor = extract_choices(df_mentor["What are your personal interests?"])

In [548]:
# make one hot df

def encode_onehot(df_input, df_output):
    count = 0
    for answer in df_output.columns:
        for column in df_input.columns:
            for result in df_input[column]:
                if answer in result: 
                    df_output.loc[df_output.index[count], answer] = 1           #.iloc[:,2:3]               
                                       
                if count == (len(df_input.index)-1):
                    count = 0
                else:
                    count += 1
    df_output = df_output.fillna(0)
    return df_output

### 1.1.1. Mentor Question 1: What are you most likely to give in this mentoring engagement?

This section creates a onehot encoded dataframe of the answers given by the mentors to this question.

The responses were classifies into 4 main types: 
* **General Advice**
* **Specific Advice**
* **Day to Day Advice**
* **Own Advice**

The following responses were edited, due to their multiple meanings:
* **All or any of the above** --> matches all the 4 response types
* **I'm flexible and will adapt to my mentee's goals** --> matches all the 4 response types

The following responses (recorded in the "others" field) were merged as following:
* **Contact Center knowledge transfer** -->  matches *Specific Advice*
* **how to become a trusted advisor and support customer with strategic planning** --> matches *Own Advice*
* **Sharing my own career journey and lessons learnt** --> matches *Own Advice*
* **An out of the box angle back on a situation.** --> matches *Own Advice*

In [549]:
df_give_mentor = encode_onehot(df_mentor, pd.DataFrame([], index=df_mentor.index, columns=give_mentor))

In [550]:
df_give_mentor.head()

,I'm flexible and will adapt to my mentee's goals,an out of the box angle back on a situation.,General strategic career advice depending on mentee goals,Sharing my own career journey and lessons learnt,how to become a trusted advisor and support customer with strategic planning,Specific career advice to prepare for next role,Tactical day to day situations & problem solving,all or any of the above
Name,,,,,,,,
22341,1,0,0,0,0,0,0,0
22342,1,0,0,0,0,0,0,0
22343,1,0,0,1,0,0,0,0
22344,1,0,0,0,0,0,0,0
22345,1,0,1,0,0,0,0,0


In [551]:
# function to combine multiple onehot encoded columns into one and optionally adding to new df
def combine_cols(df, combine_cols, add_to_df="no", combine_name=None):
    for col in combine_cols:
        if col == combine_cols[0]:
            try:
                result_list = list(df[col])
            except:
                result_list = [0] * len(df)
        else:
            try:
                result_list = [x + y for x, y in zip(result_list, list(df[col]))]
            except:
                continue
    result_list = [1 if x>1 else x for x in result_list]
            
    if type(add_to_df) == pd.core.frame.DataFrame:
        add_to_df[combine_name] = result_list
        add_to_df.set_index(df.index.values, inplace=True)
        return add_to_df
    else:
        return result_list

In [552]:
df_give_mentor_final = pd.DataFrame()

# combining the columns
cols = ["General strategic career advice depending on mentee goals", 
        "all or any of the above", 
        "I'm flexible and will adapt to my mentee's goals"]
df_give_mentor_final = combine_cols(df_give_mentor, cols, add_to_df=df_give_mentor_final, combine_name="general_advice")

# Specific advice
cols = ["all or any of the above", # own answer
        "I'm flexible and will adapt to my mentee's goals",
        "Specific career advice to prepare for next role"]
df_give_mentor_final = combine_cols(df_give_mentor, cols, add_to_df=df_give_mentor_final, combine_name="specific_advice")

# Advice of day to day situations & problem solving
cols = ["Tactical day to day situations & problem solving", 
        "all or any of the above", 
        "I'm flexible and will adapt to my mentee's goals"]
df_give_mentor_final = combine_cols(df_give_mentor, cols, add_to_df=df_give_mentor_final, combine_name="day2day_advice")

# Own experience
cols = ["how to become a trusted advisor and support customer with strategic planning ", # own answer
        "Sharing my own career journey and lessons learnt", 
        "an out of the box angle back on a situation. ", # own answer
        "all or any of the above", 
        "I'm flexible and will adapt to my mentee's goals"]
df_give_mentor_final = combine_cols(df_give_mentor, cols, add_to_df=df_give_mentor_final, combine_name="own_advice")


This is the final one hot encoded dataframe of the answers to Mentor Question 1:

In [553]:
df_give_mentor_final.head()

,general_advice,specific_advice,day2day_advice,own_advice
22341,1,1,1,1
22342,1,1,1,1
22343,1,1,1,1
22344,1,1,1,1
22345,1,1,1,1


### 1.1.2. Mentor Question 2: What are you general strengths?

This section creates a onehot encoded dataframe of the answers given by the mentors to this question.

The following responses (recorded in the "others" field) were merged as following:
* **Social Media** -->  matches *Creativity/Innovation*
* **Rapport Building** --> matches *Customer Interaction* and *Crucial Conversations*
* **Team building** --> matches *Empowerment* and *Recognition*
* **Collaboration** --> matches *Productivity*
* **Accountability** --> matches *Recognition*

In [554]:
df_strengths_mentor = encode_onehot(df_mentor, pd.DataFrame([], index=df_mentor.index, columns=strengths_mentor))

In [555]:
df_strengths_mentor.head()

,Competitive strategy,Decision making,Influencing,Creativity / innovation,Collaboration,Cross-architecture,Effective listening,Story telling,Rapport Building,Critical thinking,...,Providing feedback,Team building,Communications (public speaking),Coaching,Crucial conversations,Social Media,Delegating,Finance,Productivity,Goal setting
Name,,,,,,,,,,,,,,,,,,,,,
22341,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
22342,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
22343,0,1,0,1,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
22344,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22345,0,0,0,1,0,0,0,0,0,1,...,0,1,0,1,0,0,0,0,0,0


In [556]:
df_strengths_mentor_final = pd.DataFrame()
exclude = []

# Customer interaction & Rapport Building
cols = ["Customer interaction", 
        "Rapport Building"] # own answer
df_strengths_mentor_final = combine_cols(df_strengths_mentor, cols, add_to_df=df_strengths_mentor_final, combine_name="Customer interaction")
exclude.append(cols)

# Crucial Conversations & Rapport Building
cols = ["Crucial conversations", 
        "Rapport Building"] # own answer
df_strengths_mentor_final = combine_cols(df_strengths_mentor, cols, add_to_df=df_strengths_mentor_final, combine_name="Crucial conversations")
exclude.append(cols)

exclude = [item for sublist in exclude for item in sublist]

In [557]:
for column in df_strengths_mentor.columns: 
    if column not in exclude:
        df_strengths_mentor_final[column] = list(df_strengths_mentor[column])

In [558]:
df_strengths_mentor_final.head()

,Customer interaction,Crucial conversations,Competitive strategy,Decision making,Influencing,Creativity / innovation,Collaboration,Cross-architecture,Effective listening,Story telling,...,Time management,Providing feedback,Team building,Communications (public speaking),Coaching,Social Media,Delegating,Finance,Productivity,Goal setting
22341,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
22342,1,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
22343,0,0,0,1,0,1,1,0,0,0,...,0,0,0,1,0,1,0,0,0,1
22344,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22345,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


### 1.1.2. Mentor Question 3: What are your personal interests?

This section creates a onehot encoded dataframe of the answers given by the mentors to this question.

The following responses (recorded in the "others" field) were merged as following:
* **My work. I know this sounds pleasing but it is the truth** -->  no match
* **Ranching** --> matches *Pets*
* **Car racing** --> matches *Extreme sports (sky diving)*
* **Creating technology video** --> matches *Tech hobbies* and *Photography*
* **Cycling** --> matches *Individual sports (golf, running)*
* **Automobile and Motorcycles collecting and building** --> matches *Puzzles & games*
* **yoga** --> matches *Individual sports (golf, running)*
* **Golf** --> matches *Individual sports (golf, running)*
* **Boating** --> matches *Individual sports (golf, running)*
* **Flying** --> matches *Travel*
* **Skiing and Scuba Diving** --> matches *Extreme sports (sky diving)* and *Individual sports (golf, running)*
* **karate** --> matches *Individual sports (golf, running)*
* **Diving** --> matches *Extreme sports (sky diving)*

In [559]:
df_interests_mentor = encode_onehot(df_mentor, pd.DataFrame([], index=df_mentor.index, columns=interests_mentor))

In [560]:
df_interests_mentor.columns

Index(['karate', 'Design', 'Boating',
       'Automobile and Motorcycles collecting and building',
       'Extreme sports (sky diving)', 'Meditation', 'Flying',
       'My work. I know this sounds pleasing but it is the truth', 'Dance',
       'Photography', 'Car racing', 'Skiing and Scuba Diving', 'Art',
       'Culture', 'Team sports (soccer, football)', 'Chess', 'Puzzles & games',
       'Shopping/fashion', 'Ranching', 'Creating technology video', 'Family',
       'Individual sports (golf, running)', 'Working out', 'Volunteering',
       'Religion', 'Politics', 'Tech hobbies', 'yoga', 'Golf',
       'Social networking', 'Travel', 'Wine tasting', 'Cycling', 'Gardening',
       'Music', 'Diving', 'Reading', 'Cooking', 'Pets'],
      dtype='object')

In [561]:
df_interests_mentor_final = pd.DataFrame()

# Pets & Ranching
cols = ["Pets", 
        "Ranching"] # own answer
df_interests_mentor_final = combine_cols(df_interests_mentor, cols, add_to_df=df_interests_mentor_final, combine_name="Pets")
exclude.append(cols)

# Extreme sports (sky diving) & Car racing & Skiing and Scuba Diving & Diving
cols = ["Extreme sports (sky diving)",
        "Car racing",
        "Skiing and Scuba Diving",# own answer
        "Diving"] # own answer
df_interests_mentor_final = combine_cols(df_interests_mentor, cols, add_to_df=df_interests_mentor_final, combine_name="Extreme sports (sky diving)")
exclude.append(cols)

# Tech Hobbies & Creating technology video
cols = ["Tech hobbies",
        "Creating technology video"] # own answer
df_interests_mentor_final = combine_cols(df_interests_mentor, cols, add_to_df=df_interests_mentor_final, combine_name="Tech Hobbies")
exclude.append(cols)

# Photography & Creating technology video &
cols = ["Photography",
        "Creating technology video"] # own answer
df_interests_mentor_final = combine_cols(df_interests_mentor, cols, add_to_df=df_interests_mentor_final, combine_name="Photography")
exclude.append(cols)

# Individual sports (golf, running) & Creating technology video &
cols = ["Individual sports (golf, running)",
        "Cycling", # own answer
        "yoga", # own answer
        "Golf", # own answer
        "Boating", # own answer
        "Skiing and Scuba Diving", # own answer
        "karate"] # own answer
df_interests_mentor_final = combine_cols(df_interests_mentor, cols, add_to_df=df_interests_mentor_final, combine_name="Individual sports (golf, running)")
exclude.append(cols)

# Puzzles & games & Automobile and Motorcycles collecting and building
cols = ["Puzzles & games",
        "Automobile and Motorcycles collecting and building"] # own answer
df_interests_mentor_final = combine_cols(df_interests_mentor, cols, add_to_df=df_interests_mentor_final, combine_name="Puzzles & games")
exclude.append(cols)

# Travel & Flying
cols = ["Travel",
        "Flying"] # own answer
df_interests_mentor_final = combine_cols(df_interests_mentor, cols, add_to_df=df_interests_mentor_final, combine_name="Travel")
exclude.append(cols)

exclude = [item for sublist in exclude for item in sublist]

In [562]:
for column in df_interests_mentor.columns: 
    if column not in exclude:
        df_interests_mentor_final[column] = list(df_interests_mentor[column])

In [563]:
df_interests_mentor_final.head()

,Pets,Extreme sports (sky diving),Tech Hobbies,Photography,"Individual sports (golf, running)",Puzzles & games,Travel,Design,Meditation,Dance,...,Working out,Volunteering,Religion,Politics,Social networking,Wine tasting,Gardening,Music,Reading,Cooking
22341,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
22342,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
22343,0,0,0,0,1,0,0,1,0,0,...,1,1,0,0,0,0,1,0,0,1
22344,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22345,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0


### 1.1.4. Combine all three mentor dataframes into one

The dataframes containing the Mentor's andwers "advice to give", "strengths" and "personal interests" are combined into one single one-hot encoded dataframe.

In [564]:
df_onehot_mentor = pd.concat([df_give_mentor_final, df_strengths_mentor_final,df_interests_mentor_final], axis=1)
df_onehot_mentor = df_onehot_mentor.reindex(sorted(df_onehot_mentor.columns), axis=1)

In [565]:
df_onehot_mentor.head()

,Accountability,Art,Change management,Chess,Coaching,Collaboration,Communications (public speaking),Competitive strategy,Cooking,Creativity / innovation,...,Transition career path,Travel,Volunteering,Wine tasting,Work / Life balance,Working out,day2day_advice,general_advice,own_advice,specific_advice
22341,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,1,1
22342,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,1,1,1,1
22343,1,0,0,0,0,1,1,0,1,1,...,0,0,1,0,1,1,1,1,1,1
22344,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,1,1,1
22345,0,1,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,1,1,1,1


## 1.2. Mentee data

Now we repeat the process with the Mentee data.

In [566]:
# read in mentee data
df_mentee_raw = pd.read_excel(r'FY21 - Mentee Interest - Chairman Club & Hall of Fame Mentoring Program.xlsx',sheet_name='From_a')
df_mentee_raw = pd.DataFrame(df_mentee_raw).set_index('Name', drop=True)
df_mentee_raw.shape

(79, 23)

In [567]:
df_mentee_raw = drop_nan_cols(df_mentee_raw)
df_mentee_raw.shape

# no columns had all NaNs

(79, 23)

In [568]:
df_mentee = df_mentee_raw[["What are you most likely to want out of this mentoring engagement?",
               "What are the strengths you would like to further develop?",
               "What are your personal interests?"]]

In [569]:
want_mentee = extract_choices(df_mentee["What are you most likely to want out of this mentoring engagement?"])
strengths_mentee = extract_choices(df_mentee["What are the strengths you would like to further develop?"])
interests_mentee = extract_choices(df_mentee["What are your personal interests?"])

### 1.2.1. Mentee Question 1: What are you most likely to want out of this mentoring engagement?

This section creates a onehot encoded dataframe of the answers given by the mentees to this question.

The responses were classifies into 4 main types: 
* **General Advice**
* **Specific Advice**
* **Day to Day Advice**
* **Own Advice**

The following responses (recorded in the "others" field) were edited in order to not skew the people matching. If a mentee chooses this, he/she will always be more different than each of the mentors, since no mentor will meet this creteria. 
* **Form a strategic plan on how to become the best (or the mindset that will help me do)**
* **situational experiences (wins and losses) Want to hear their perspective on how to be a good team player Want to hear their thoughts on market changes and where tech, energy and agriculture are headed.**

All responses were add to the **Own Advice** type, since otherwise the data would aswell be skewed. Now it has no impact on the mentee side of data.

In [570]:
df_want_mentee = encode_onehot(df_mentee, pd.DataFrame([], index=df_mentee.index, columns=want_mentee))

In [571]:
df_want_mentee.head()

,General strategic career advice,Form a strategic plan on how to become the best (or the mindset that will help me do),tactical day to day situations & problem solving,"Looking to hear about how they handle pressure, situational experiences (wins and losses) Want to hear their perspective on how to be a good team player Want to hear their thoughts on market changes and where tech, energy and agriculture are headed.",Specific career advice to prepare for next role,How to build better relationships with customers and partners /,Tactical day to day situations & problem solving,How one could maximize the talent and potential,Anything that they wish they had known in my situation,Know/uncover my strengths and differentiators
Name,,,,,,,,,,
12345,1,0,0,0,0,0,0,0,0,0
12346,1,0,0,0,0,0,1,0,0,0
12347,1,0,0,0,0,0,0,0,0,0
12348,1,0,0,0,0,0,0,0,0,0
12349,1,0,0,0,1,0,1,0,0,0


In [572]:
df_want_mentee_final = pd.DataFrame()
exclude = []

# combining the columns
cols = ["Form a strategic plan on how to become the best (or the mindset that will help me do)", 
        "General strategic career advice", 
        "Looking to hear about how they handle pressure, situational experiences (wins and losses) Want to hear their perspective on how to be a good team player Want to hear their thoughts on market changes and where tech, energy and agriculture are headed. "]
df_want_mentee_final = combine_cols(df_want_mentee, cols, add_to_df=df_want_mentee_final, combine_name="general_advice")
exclude.append(cols)

# Specific advice
cols = ["Form a strategic plan on how to become the best (or the mindset that will help me do)", # own answer
        "Specific career advice to prepare for next role", 
        "Looking to hear about how they handle pressure, situational experiences (wins and losses) Want to hear their perspective on how to be a good team player Want to hear their thoughts on market changes and where tech, energy and agriculture are headed. ",
        "How one could maximize the talent and potential",
        "How to build better relationships with customers and partners / "]
df_want_mentee_final = combine_cols(df_want_mentee, cols, add_to_df=df_want_mentee_final, combine_name="specific_advice")
exclude.append(cols)

# Advice of day to day situations & problem solving
cols = ["Form a strategic plan on how to become the best (or the mindset that will help me do)", # own answer
        "Tactical day to day situations & problem solving", 
        "tactical day to day situations & problem solving",
        "Looking to hear about how they handle pressure, situational experiences (wins and losses) Want to hear their perspective on how to be a good team player Want to hear their thoughts on market changes and where tech, energy and agriculture are headed. "]
df_want_mentee_final = combine_cols(df_want_mentee, cols, add_to_df=df_want_mentee_final, combine_name="day2day_advice")
exclude.append(cols)

# Own experience
cols = ["Form a strategic plan on how to become the best (or the mindset that will help me do)", 
        "General strategic career advice", 
        "Specific career advice to prepare for next role",
        "Tactical day to day situations & problem solving", 
        "tactical day to day situations & problem solving",
        "Looking to hear about how they handle pressure, situational experiences (wins and losses) Want to hear their perspective on how to be a good team player Want to hear their thoughts on market changes and where tech, energy and agriculture are headed. ",
        "Anything that they wish they had known in my situation",
        "Know/uncover my strengths and differentiators"]
df_want_mentee_final = combine_cols(df_want_mentee, cols, add_to_df=df_want_mentee_final, combine_name="own_advice")
exclude.append(cols)

exclude = [item for sublist in exclude for item in sublist]

In [573]:
for column in df_want_mentee.columns: 
    if column not in df_want_mentee_final.columns and column not in exclude:
        df_want_mentee_final[column] = list(df_want_mentee[column])

In [574]:
df_want_mentee_final.head()

,general_advice,specific_advice,day2day_advice,own_advice
12345,1,0,0,1
12346,1,0,1,1
12347,1,0,0,1
12348,1,0,0,1
12349,1,1,1,1


### 1.2.2. Mentee Question 2: What are the strengths you'd like to further develop?

This section creates a onehot encoded dataframe of the answers given by the mentees to this question.

In [575]:
# with ID column
df_strengths_mentee_final = encode_onehot(df_mentee, pd.DataFrame([], index=df_mentee.index, columns=strengths_mentee))

#df_strengths_mentee_final = encode_onehot(df_mentee, strengths_mentee)

In [576]:
df_strengths_mentee_final.head()

,Competitive strategy,Decision making,Influencing,Creativity / innovation,Effective listening,Cross-architecture,Collaboration,Story telling,Critical thinking,Work / Life balance,...,Team building,Crucial conversations,Communications (public speaking),Coaching,Social Media,Delegating,Finance,Productivity,Goal setting,Personal Brand
Name,,,,,,,,,,,,,,,,,,,,,
12345,0,1,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
12346,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347,0,0,0,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
12348,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,1,0
12349,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


### 1.2.3. Mentee Question 3: What are your personal interests?

This section creates a onehot encoded dataframe of the answers given by the mentees to this question.

The following responses (recorded in the "others" field) were merged as following:
* **Pen and Paper role-playing games (e.g. Dungeons & Dragons)** -->  matches *Puzzles & Games*
* **Programming (just beginning to learn!)** --> matches *Tech hobbies*
* **Coffee** --> matches *Cooking*

In [577]:
df_interests_mentee = encode_onehot(df_mentee, pd.DataFrame([], index=df_mentee.index, columns=interests_mentee))

In [578]:
df_interests_mentee.columns

Index(['Design', 'Psychology ', 'Extreme sports (sky diving)', 'Meditation',
       'Dance', 'Photography', 'Coffee', 'Art', 'Culture',
       'Team sports (soccer, football)', 'Chess', 'Shopping/fashion',
       'Puzzles & games', 'Family', 'Individual sports (golf, running)',
       'Working out', 'Volunteering', 'Politics', 'Religion', 'Tech hobbies',
       'Programming (just beginning to learn!)', 'Social networking', 'Travel',
       'Wine tasting', 'Stock market', 'Gardening', 'Music', 'Reading',
       'Pen and Paper role-playing games (e.g. Dungeons & Dragons)', 'Cooking',
       'Pets'],
      dtype='object')

In [579]:
df_interests_mentee_final = pd.DataFrame()
exclude = []

# Pets & Ranching
cols = ["Puzzles & games", 
        "Pen and Paper role-playing games (e.g. Dungeons & Dragons)"] # own answer
df_interests_mentee_final = combine_cols(df_interests_mentee, cols, add_to_df=df_interests_mentee_final, combine_name="Puzzles & games")
exclude.append(cols)

# Extreme sports (sky diving) & Car racing & Skiing and Scuba Diving & Diving
cols = ["Cooking",
        "Coffee"] # own answer
df_interests_mentee_final = combine_cols(df_interests_mentee, cols, add_to_df=df_interests_mentee_final, combine_name="Cooking")
exclude.append(cols)

# Tech Hobbies & Creating technology video
cols = ["Tech hobbies",
        "Programming (just beginning to learn!)"] # own answer
df_interests_mentee_final = combine_cols(df_interests_mentee, cols, add_to_df=df_interests_mentee_final, combine_name="Tech Hobbies")
exclude.append(cols)

# Stock market added to Finance strengths
for mentee in df_interests_mentee.index:
    try:
        if df_interests_mentee["Stock market"][mentee] == 1:
            df_strengths_mentee_final["Finance"][mentee] = 1
        exclude.append(["Stock market"])
    except:
        continue

# Stock market added to Influencing and Negotiating strengths
for mentee in df_interests_mentee.index:
    try:
        if df_interests_mentee["Psychology "].loc[mentee] == 1:
            df_strengths_mentee_final["Influencing"].loc[mentee] = 1
            df_strengths_mentee_final["Negotiating"].loc[mentee] = 1
        exclude.append(["Psychology "])
    except:
        continue

exclude = [item for sublist in exclude for item in sublist]

In [580]:
for column in df_interests_mentee.columns: 
    if column not in exclude:
        df_interests_mentee_final[column] = list(df_interests_mentee[column])

In [581]:
df_interests_mentee_final.columns

Index(['Puzzles & games', 'Cooking', 'Tech Hobbies', 'Design',
       'Extreme sports (sky diving)', 'Meditation', 'Dance', 'Photography',
       'Art', 'Culture', 'Team sports (soccer, football)', 'Chess',
       'Shopping/fashion', 'Family', 'Individual sports (golf, running)',
       'Working out', 'Volunteering', 'Politics', 'Religion',
       'Social networking', 'Travel', 'Wine tasting', 'Gardening', 'Music',
       'Reading', 'Pets'],
      dtype='object')

### 1.2.4. Combine all three mentee dataframes into one:

In [582]:
df_onehot_mentee = pd.concat([df_want_mentee_final, df_strengths_mentee_final, df_interests_mentee_final], axis=1)
df_onehot_mentee = df_onehot_mentee.reindex(sorted(df_onehot_mentee.columns), axis=1)

In [583]:
df_onehot_mentee.head()

,Accountability,Art,Change management,Chess,Coaching,Collaboration,Communications (public speaking),Competitive strategy,Cooking,Creativity / innovation,...,Transition career path,Travel,Volunteering,Wine tasting,Work / Life balance,Working out,day2day_advice,general_advice,own_advice,specific_advice
12345,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,1,1,0
12346,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,1,1,1,0
12347,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,0
12348,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,1,1,0
12349,0,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,1,1,1,1,1


## 1.3. Check the dataframes for errors

In [584]:
df_onehot_mentee.columns

Index(['Accountability', 'Art', 'Change management', 'Chess', 'Coaching',
       'Collaboration', 'Communications (public speaking)',
       'Competitive strategy', 'Cooking', 'Creativity / innovation',
       'Critical thinking', 'Cross-architecture', 'Crucial conversations',
       'Culture', 'Customer interaction', 'Dance', 'Decision making',
       'Delegating', 'Design', 'Design thinking', 'Effective listening',
       'Empowerment', 'Extreme sports (sky diving)', 'Family', 'Finance',
       'Gardening', 'Goal setting', 'Individual sports (golf, running)',
       'Influencing', 'Meditation', 'Music', 'Negotiating', 'Personal Brand',
       'Pets', 'Photography', 'Politics', 'Prioritization', 'Productivity',
       'Providing feedback', 'Puzzles & games', 'Reading', 'Recognition',
       'Religion', 'Shopping/fashion', 'Social Media', 'Social networking',
       'Story telling', 'Team building', 'Team sports (soccer, football)',
       'Tech Hobbies', 'Technical knowledge', 'Time m

In [585]:
df_onehot_mentor.columns

Index(['Accountability', 'Art', 'Change management', 'Chess', 'Coaching',
       'Collaboration', 'Communications (public speaking)',
       'Competitive strategy', 'Cooking', 'Creativity / innovation',
       'Critical thinking', 'Cross-architecture', 'Crucial conversations',
       'Culture', 'Customer interaction', 'Dance', 'Decision making',
       'Delegating', 'Design', 'Design thinking', 'Effective listening',
       'Empowerment', 'Extreme sports (sky diving)', 'Family', 'Finance',
       'Gardening', 'Goal setting', 'Individual sports (golf, running)',
       'Influencing', 'Meditation', 'Music', 'Negotiating', 'Personal Brand',
       'Pets', 'Photography', 'Politics', 'Prioritization', 'Productivity',
       'Providing feedback', 'Puzzles & games', 'Reading', 'Recognition',
       'Religion', 'Shopping/fashion', 'Social Media', 'Social networking',
       'Story telling', 'Team building', 'Team sports (soccer, football)',
       'Tech Hobbies', 'Technical knowledge', 'Time m

In [586]:
# same columns
for cols in df_onehot_mentee.columns:
    if cols not in df_onehot_mentor.columns:
        print(cols)

In [587]:
for cols in df_onehot_mentor.columns:
    if cols not in df_onehot_mentee.columns:
        print(cols)

In [588]:
# add missin columns
for column in df_onehot_mentee.columns:
    if column not in df_onehot_mentor.columns:
        df_onehot_mentor[column] = 0
        
for column in df_onehot_mentor.columns:
    if column not in df_onehot_mentee.columns:
        df_onehot_mentee[column] = 0

In [589]:
df_onehot_mentee

,Accountability,Art,Change management,Chess,Coaching,Collaboration,Communications (public speaking),Competitive strategy,Cooking,Creativity / innovation,...,Transition career path,Travel,Volunteering,Wine tasting,Work / Life balance,Working out,day2day_advice,general_advice,own_advice,specific_advice
12345,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,1,1,0
12346,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,1,1,1,0
12347,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,0
12348,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,1,1,0
12349,0,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12419,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
12420,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,1,1,1,1,1
12421,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
12423,0,0,1,0,0,0,1,1,0,0,...,1,1,0,1,0,0,0,1,1,0


# 2. Calculating Similarity of Users

In [590]:
# calculate the Jaccard distance between each mentee and each mentor
# 1-Jaccard distance = similarity between pairs
ary = scipy.spatial.distance.cdist(df_onehot_mentor.iloc[:,1:], df_onehot_mentee.iloc[:,1:], metric='jaccard')

people_similarity = pd.DataFrame(ary, columns=df_onehot_mentee.index.values, index=df_onehot_mentor.index.values)
people_similarity.head()

# mentees in columns (start 1), mentors in rows (start with 2)

,12345,12346,12347,12348,12349,12350,12351,12352,12353,12354,...,12414,12415,12416,12417,12418,12419,12420,12421,12423,12424
22341,0.789474,0.809524,0.800000,0.863636,0.687500,0.666667,0.588235,0.777778,0.703704,0.687500,...,0.705882,0.666667,0.812500,0.733333,0.800000,0.812500,0.625000,0.642857,0.809524,0.733333
22342,0.705882,0.736842,0.850000,0.857143,0.666667,0.812500,0.647059,0.769231,0.827586,0.666667,...,0.687500,0.733333,0.800000,0.800000,0.785714,0.800000,0.687500,0.714286,0.800000,0.800000
22343,0.758621,0.774194,0.766667,0.774194,0.740741,0.862069,0.806452,0.685714,0.628571,0.827586,...,0.750000,0.777778,0.814815,0.857143,0.851852,0.933333,0.703704,0.814815,0.882353,0.857143
22344,0.777778,0.736842,0.789474,0.909091,0.666667,0.812500,0.466667,0.769231,0.740741,0.666667,...,0.764706,0.642857,0.800000,0.714286,0.785714,0.800000,0.600000,0.714286,0.736842,0.800000
22345,0.826087,0.681818,0.782609,0.884615,0.750000,0.909091,0.727273,0.843750,0.700000,0.750000,...,0.818182,0.857143,0.850000,0.722222,0.900000,0.850000,0.700000,0.789474,0.791667,0.904762


In [591]:
# check which pair has the highest similarity in the dataset
a, b = people_similarity.stack().idxmin()
print(people_similarity.loc[[a], [b]])

          12351
22399  0.307692


In [592]:
people_similarity[12345].nsmallest(len(people_similarity))

22342    0.705882
22423    0.705882
22415    0.705882
22397    0.714286
22374    0.722222
           ...   
22403    0.913043
22368    0.916667
22388    0.941176
22407    0.944444
22402    0.944444
Name: 12345, Length: 85, dtype: float64

# 3. Assigning Mentor - Mentee Pairs

In [593]:
# function that assigns the list of mentors ranked by similarity to each mentee
def create_cluster(df):
    clusters = {}
    for mentee in df.columns:
        try:
            mentee_cluster = df[mentee].nsmallest(len(df))
            data = {mentee : list(mentee_cluster.index)}
            clusters.update(data)
        except:
            continue
    return clusters

In [594]:
people_cluster = create_cluster(people_similarity)
people_cluster

# for mentee 12345, the most similar mentor is 22342

{12345: [22342,
  22423,
  22415,
  22397,
  22374,
  22381,
  22359,
  22384,
  22343,
  22357,
  22395,
  22412,
  22379,
  22424,
  22422,
  22365,
  22344,
  22347,
  22421,
  22398,
  22405,
  22353,
  22404,
  22408,
  22356,
  22396,
  22341,
  22358,
  22416,
  22352,
  22354,
  22362,
  22361,
  22386,
  22400,
  22360,
  22390,
  22428,
  22418,
  22345,
  22380,
  22406,
  22349,
  22373,
  22414,
  22399,
  22376,
  22346,
  22350,
  22351,
  22355,
  22410,
  22366,
  22385,
  22394,
  22375,
  22427,
  22389,
  22391,
  22425,
  22383,
  22348,
  22413,
  22417,
  22393,
  22382,
  22364,
  22426,
  22370,
  22409,
  22392,
  22369,
  22371,
  22411,
  22363,
  22372,
  22387,
  22419,
  22401,
  22367,
  22403,
  22368,
  22388,
  22407,
  22402],
 12346: [22351,
  22357,
  22349,
  22394,
  22404,
  22346,
  22423,
  22419,
  22372,
  22345,
  22352,
  22348,
  22425,
  22369,
  22413,
  22384,
  22424,
  22373,
  22412,
  22356,
  22354,
  22415,
  22410,
  22365,
  22

# 4. Filter assignments by mentee/mentor preferences
We now have a ranking of the mentors that are most similar to the respective mentees. Each mentee has a list of the mentors ranked by their similarity regarding the survey choices.

Now we consider special preferences that the mentees/mentors chose within the survey.

The order of priorities considered is as following:

1. First preference of mentee 
2. Theatre preference of mentor
3. Role preference of mentor

In [595]:
# test to check if each mentee has at least 1 metor assigned to it
def test_assignment(info="default"):
    for mentee in people_cluster:
        if len(people_cluster[mentee]) < 1:
            raise ValueError(mentee, "has less than 1 mentor assigned! Info:", info)
    print("Test passed.")

## 4.1. Prepare Data: Theatre

### 4.1.1. Mentor: Theatre

In [596]:
df_mentor_theatre = df_mentor_raw[["Would you prefer a mentee in the same theatre?", 
                                "Your theatre?"]]

# filter by all the mentors that want a mentee in the same theatre
df_mentor_theatre_filter = df_mentor_theatre[df_mentor_theatre["Would you prefer a mentee in the same theatre?"]!="No preference"]

### 4.1.2. Mentee: Theatre

In [597]:
df_mentee_theatre = df_mentee_raw[["If possible, would you prefer a mentor in the same theatre?", 
                                   "Your theatre?"]]
df_mentee_theatre_filter = df_mentee_theatre[df_mentee_theatre["If possible, would you prefer a mentor in the same theatre?"]!="No preference"]

## 4.2. Prepare Data: Role
### 4.2.1. Mentor: Role

In [599]:
df_mentor_role = df_mentor_raw[["Would you prefer a mentee in a:", 
                                "What is your title?"]]

# filter by all the mentors that want a mentee in the same theatre
df_mentor_role_filter = df_mentor_role[df_mentor_role["Would you prefer a mentee in a:"]!="No preference"]

In [600]:
def replace_roles(df, col):
    technical = ["technical", "system", "se", "Technichal"] # systems atchitect
    sales = ["account manager", "business development", "client", "sales specialist", "financial", "partner", 
             "regional director", "sales manager", "renewal", "business solutions", "regional manager", "cx",
             "bdm", "product specialist", "Collaboration Executive", "Account Specialist", "vsam", "vpam"]
    for person in df.index:
        person_role = str((df[col].loc[person])).lower()

        for tech_role in technical:
            if tech_role.lower() in person_role:
                df[col][person] = "Technical role"
        for sales_role in sales:
            if sales_role.lower() in person_role:
                df[col][person] = "Sales role"

In [601]:
replace_roles(df_mentor_role, "What is your title?")

In [602]:
def test_roles(df, col):
    if len(set(df[col])) > 2:
        print(set(df[col]))
        raise ValueError("Error! Roles other than Sales & Technical Role present in dataset.")
    else:
        print("Test passed.")

In [603]:
test_roles(df_mentor_role, "What is your title?")

Test passed.


### 4.2.2. Mentee: Role

In [480]:
df_mentee_role = df_mentee_raw[["If possible, would you prefer a mentor in a:", 
                                "What is your title?"]]
df_mentee_role_filter = df_mentee_role[df_mentee_role["If possible, would you prefer a mentor in a:"]!="No preference"]

In [481]:
replace_roles(df_mentee_role, "What is your title?")

In [482]:
set(df_mentee_role["What is your title?"])

{'Sales role', 'Technical role'}

In [483]:
test_roles(df_mentee_role, "What is your title?")

Test passed.


## 4.3. Prepare Data: Segment
### 4.3.1 Mentor: Segment

In [484]:
df_mentor_segment = df_mentor_raw[["Your segment?"]]

for person in df_mentor_segment.index:
    if df_mentor_segment["Your segment?"][person] == "Enterprise Networking":
        df_mentor_segment["Your segment?"][person] = "Ent. Networking"

In [485]:
set(df_mentor_segment["Your segment?"])

{' Enterprise, commercial, public sector',
 'AppDynamics',
 'CGEM',
 'CGEM Enterprise',
 'CGEM Premier Global accounts',
 'CX',
 'Commercial',
 'Defence',
 'Ent. Networking',
 'Enterprise',
 'FED',
 'GES/CX',
 'GVSE',
 'GVSS',
 'Global Specialist Organization ',
 'Global Specialists Org',
 'Partner',
 'Public Sector',
 'Public Sector (Local and State Gov)',
 'Public Service',
 'Public sector',
 'SLED',
 'Service Provider',
 'Specialist, AppDynamics'}

### 4.3.2. Mentee: Segment

In [486]:
col = "If possible, which segment would you like your mentor to be from?"
df_mentee_segment = df_mentee_raw[[col]]
df_mentee_segment_filter = df_mentee_segment[df_mentee_segment[col]!="No preference"]
df_mentee_segment_filter = df_mentee_segment_filter[df_mentee_segment_filter[col]!="No Preference"]
df_mentee_segment_filter = df_mentee_segment_filter[df_mentee_segment_filter[col]!="Any"]
df_mentee_segment_filter = df_mentee_segment_filter[df_mentee_segment_filter[col]!="Enterprise, SP, Commercial, SLED, FED all work  "]

In [487]:
set(df_mentee_segment_filter["If possible, which segment would you like your mentor to be from?"])

{'CDA - Project Management',
 'Commercial',
 'DC',
 'Enterprise',
 'FED',
 'I still need to learn the above segments',
 'Partner',
 'SLED',
 'SP or CX',
 'Service Provider',
 'worked in multiple segments before'}

## 4.4. Filter by: First Priority Preference of Mentee

In [488]:
# prepare dataset
df_mentee_prio = df_mentee_raw[["If possible, what is your first priority in matching you with a mentor?"]]
df_mentee_prio = df_mentee_prio[df_mentee_prio["If possible, what is your first priority in matching you with a mentor?"]!="No preference"]
df_mentee_prio = df_mentee_prio[df_mentee_prio["If possible, what is your first priority in matching you with a mentor?"]!="Strengths you seek to develop"]
df_mentee_prio = df_mentee_prio[df_mentee_prio["If possible, what is your first priority in matching you with a mentor?"]!="Personal interests"]

In [489]:
def filter_by_pref(mentee, filter_by):
    if filter_by == "Theatre":
        mentee_theatre = df_mentee_theatre["Your theatre?"].loc[mentee]
        for mentor in df_mentor_theatre.index:
            mentor_theatre = df_mentor_theatre["Your theatre?"].loc[mentor] 
            if mentee_theatre != mentor_theatre:
                try:
                    people_cluster[mentee].remove(mentor)
                except:
                    continue
    elif filter_by == "Role":
        mentee_role = df_mentee_role["What is your title?"].loc[mentee]
        for mentor in df_mentor_role.index:
            mentor_role = df_mentor_role["What is your title?"].loc[mentor]
            if mentee_role != mentor_role:
                try:
                    people_cluster[mentee].remove(mentor)
                except:
                    continue
    elif filter_by == "Segment":
        if mentee in df_mentee_segment_filter.index:
            mentee_segment = df_mentee_segment_filter["If possible, which segment would you like your mentor to be from?"].loc[mentee]
            for mentor in df_mentor_segment.index:
                mentor_segment = df_mentor_segment["Your segment?"].loc[mentor]
                if mentee == "Morgan Carroll":
                    print(mentee)
                    print(mentor_segment)
                    if "sled" not in mentor_segment.lower() and "enterprise" not in mentor_segment.lower():
                        people_cluster[mentee].remove(mentor)
                elif mentee_segment.lower() not in mentor_segment.lower():
                    try:
                        people_cluster[mentee].remove(mentor)
                    except:
                        continue
    elif filter_by == "Gender":
        mentee_gender = df_mentee_raw["Gender"].loc[mentee]
        for mentor in df_mentor_raw.index:
            mentor_gender = df_mentor_raw["Gender"].loc[mentor]
            if mentee_gender != mentor_gender:
                try:
                    people_cluster[mentee].remove(mentor)
                except:
                    continue               

In [490]:
col ="If possible, what is your first priority in matching you with a mentor?"
for mentee in df_mentee_prio.index:
    try:
        if df_mentee_prio[col].loc[mentee] == "Theatre":
            filter_by_pref(mentee, "Theatre")
        elif df_mentee_prio[col].loc[mentee] == "Role (Tech/Sales)":
            filter_by_pref(mentee, "Role")
        elif df_mentee_prio[col].loc[mentee] == "Segment":
            filter_by_pref(mentee, "Segment")
        elif df_mentee_prio[col].loc[mentee] == "Gender":
            filter_by_pref(mentee, "Gender")
    except:
        continue

In [491]:
test_assignment()

Test passed.


In [492]:
#df_mentor_raw.isnull().sum(axis = 0)

# 4.5. Filter by Mentor Preference: Theatre

In [493]:
for mentor in df_mentor_raw.index:
    if df_mentor_raw["Would you prefer a mentee in the same theatre?"].loc[mentor]== "Yes":
        mentor_theatre = df_mentor_raw["Your theatre?"].loc[mentor]
        for mentee in people_cluster:
            mentee_theatre = df_mentee_raw["Your theatre?"].loc[mentee]
            if mentor_theatre != mentee_theatre:
                try:
                    people_cluster[mentee].remove(mentor)
                except:
                    continue         
# filter out mentees/mentor if both from AMER and APJC, because of timezone difference                   
for mentor in df_mentor_raw.index:
    mentor_theatre = df_mentor_raw["Your theatre?"].loc[mentor]
    for mentee in people_cluster:
        mentee_theatre = df_mentee_raw["Your theatre?"].loc[mentee]
        if (mentor_theatre == "AMER" and mentee_theatre == "APJC") or (mentor_theatre == "APJC" and mentee_theatre == "AMER"):
            try:
                people_cluster[mentee].remove(mentor)
            except:
                continue       

In [494]:
test_assignment()

Test passed.


In [495]:
for mentee in people_cluster:
    mentee_theatre = df_mentee_raw["Your theatre?"].loc[mentee]
    if mentee_theatre == "AMER":
        for mentor in people_cluster[mentee]:
            mentor_theatre = df_mentor_raw["Your theatre?"].loc[mentor]
            if mentor_theatre == "APJC":
                raise ValueError ("Oops, AMER mentee assigned with APJC mentor.")    
    elif mentee_theatre == "APJC":
        for mentor in people_cluster[mentee]:
            mentor_theatre = df_mentor_raw["Your theatre?"].loc[mentor]
            if mentor_theatre == "AMER":
                raise ValueError ("Oops, APJC mentee assigned with AMER mentor.")    
print("Test passed.")


Test passed.


In [496]:
people_cluster

{12345: [22342,
  22423,
  22415,
  22397,
  22374,
  22359,
  22343,
  22357,
  22412,
  22395,
  22422,
  22424,
  22365,
  22398,
  22404,
  22405,
  22347,
  22408,
  22356,
  22416,
  22341,
  22352,
  22358,
  22400,
  22428,
  22406,
  22414,
  22373,
  22399,
  22349,
  22351,
  22350,
  22410,
  22346,
  22378,
  22391,
  22376,
  22389,
  22394,
  22385,
  22427,
  22425,
  22417,
  22393,
  22364,
  22382,
  22426,
  22369,
  22409,
  22371,
  22411,
  22363,
  22401,
  22419,
  22387,
  22372,
  22403,
  22388,
  22402],
 12346: [22351,
  22394,
  22404,
  22352,
  22413,
  22356,
  22415,
  22365,
  22344,
  22416,
  22406,
  22411,
  22426,
  22368,
  22399,
  22405,
  22398,
  22427,
  22374,
  22341,
  22376,
  22420,
  22370,
  22421,
  22358,
  22364,
  22377,
  22407],
 12347: [22398,
  22410,
  22352,
  22428,
  22356,
  22393,
  22401,
  22372,
  22427,
  22387,
  22404,
  22394,
  22417,
  22376,
  22400,
  22359,
  22343,
  22357,
  22395,
  22422,
  22349,
  224

# 4.6. Filter by Mentor Preference: Role

In [497]:
for mentor in df_mentor_raw.index:
    if df_mentor_raw["Would you prefer a mentee in a:"].loc[mentor]!= "No preference":
        mentor_role = df_mentor_raw["Would you prefer a mentee in a:"].loc[mentor]
        for mentee in people_cluster:
            mentee_role = df_mentee_role["What is your title?"].loc[mentee]
            if mentor_role != mentee_role:
                try:
                    people_cluster[mentee].remove(mentor)
                except:
                    continue          

In [498]:
# for mentee in people_cluster:
#     print(len(people_cluster[mentee]))

In [499]:
test_assignment()

Test passed.


# 4.7. Filter by Mentor preference: Gender

In [500]:
for mentor in df_mentor_raw.index:
    if df_mentor_raw["Would you prefer a mentee of the same gender?"].loc[mentor]== "Yes":
        mentor_gender = df_mentor_raw["Gender"].loc[mentor]
        for mentee in people_cluster:
            mentee_gender = df_mentee_raw["Gender"].loc[mentee]
            if mentor_gender != mentee_gender:
                try:
                    people_cluster[mentee].remove(mentor)
                except:
                    continue          

In [501]:
test_assignment()

Test passed.


In [502]:
people_cluster

{12345: [22342,
  22423,
  22415,
  22397,
  22374,
  22359,
  22343,
  22412,
  22395,
  22422,
  22424,
  22365,
  22404,
  22347,
  22356,
  22416,
  22341,
  22352,
  22358,
  22400,
  22428,
  22406,
  22414,
  22373,
  22399,
  22349,
  22351,
  22350,
  22410,
  22346,
  22378,
  22376,
  22389,
  22385,
  22427,
  22417,
  22393,
  22364,
  22382,
  22426,
  22369,
  22409,
  22371,
  22411,
  22363,
  22401,
  22419,
  22387,
  22372,
  22403,
  22388,
  22402],
 12346: [22351,
  22404,
  22352,
  22413,
  22356,
  22415,
  22365,
  22344,
  22416,
  22406,
  22411,
  22426,
  22399,
  22427,
  22374,
  22341,
  22376,
  22420,
  22358,
  22364,
  22377,
  22407],
 12347: [22410,
  22352,
  22428,
  22356,
  22393,
  22401,
  22372,
  22427,
  22387,
  22404,
  22417,
  22376,
  22400,
  22359,
  22395,
  22422,
  22349,
  22412,
  22365,
  22351,
  22378,
  22385,
  22423,
  22416,
  22411,
  22341,
  22374,
  22403,
  22426,
  22409,
  22363,
  22406,
  22424,
  22373,
  223

# 4.8. Filter by Mentor specific preferences

In [503]:
for mentor in df_mentor_raw.index:
    if mentor == "22424":
        print(mentor)
        for mentee in people_cluster:
            mentee_country = df_mentee_raw["Your country of residence"].loc[mentee]
            print(mentee_country.lower())
            if mentee_country.lower() != "germany":
                try:
                    people_cluster[mentee].remove(mentor)
                    print("removed")
                except:
                    print("could not remove")
                    continue     
            else:
                print(people_cluster[mentee])

In [504]:
for mentee in people_cluster:
    if "22424" in people_cluster[mentee]:
        mentee_country = df_mentee_raw["Your country of residence"].loc[mentee]
        print(mentee_country)
        if not mentee_country.lower() == "germany":
            raise ValueError("Specif assignment failed.")
        else:
            print("Test passed.")

# 6. Filter if a Mentor has a Mentee in Mind

In [506]:
df_mentor_inmind = pd.DataFrame(df_mentor_raw["Let us know if you already have a CSAP mentee in mind"])

In [507]:
df_mentor_inmind = df_mentor_inmind[df_mentor_inmind['Let us know if you already have a CSAP mentee in mind'].notna()] 

In [510]:
pairs = {}
for mentee in df_mentee_raw.index:
    for answer in list(df_mentor_raw["Let us know if you already have a CSAP mentee in mind"]):
        if not pd.isnull(answer):
            if str(mentee) in str(answer):
                mentor = df_mentor_raw.index[df_mentor_raw["Let us know if you already have a CSAP mentee in mind"] == answer]
                pairs[mentee] = mentor[0]  

In [512]:
pairs

#mentor 22406 answered he/whe has mentee 12403 in mind

{12403: 22406}

Replace the metee in mind in the people cluster distionary:

In [515]:
for mentee_mind in pairs:
    mentor_mind = pairs[mentee_mind]
    for mentee in people_cluster:
        #print(mentee)
        if mentor_mind in people_cluster[mentee]:
            people_cluster[mentee].remove(mentor_mind)
        if mentee == mentee_mind:
            people_cluster[mentee].insert(0, mentor_mind)

In [516]:
test_assignment()

Test passed.


In [517]:
# test if only the mentee in "pairs" has the mentor in "pairs" assigned
def test_mentor_mind():
    for mentee in people_cluster:
        for mentor in people_cluster[mentee]:
            if mentor in pairs.values():
                if mentee not in pairs.keys():
                    print("Error for mentee:", mentee, "and mentor:", mentor)
    print("Test passed.")

In [518]:
test_mentor_mind()

Test passed.


In [519]:
people_cluster_unique = {}
for mentee in people_cluster.keys():
    people_cluster_unique[mentee] = people_cluster[mentee][0:10]

In [520]:
##### do not touch
def sort_unique(cluster, max_rounds=60, method="limited"):
    i = 0
    j = 0
    count = []
    for number in range(0,max_rounds):
        count.append(0)
    rounds = 0
    mentees = list(cluster.keys())
    while rounds < max_rounds:
        while i < len(cluster):
            while j < len(cluster):
                if j == i:
                    j+=1
                else:
                    to_compare = list(zip(cluster[mentees[i]], cluster[mentees[j]]))[0]
                    if to_compare[0] == to_compare[1]:
                        force_position = None
                        if mentees[i] in list(pairs.keys()):
                            force_position = 0
                        if mentees[j] in list(pairs.keys()):
                            force_position = 1
                        count[rounds] += 1
                        if force_position == None:
                            position = random.randint(0, 1)
                        else:
                            position = force_position
                            
                        if rounds > 1:
                            print(count[rounds-1])
                            if count[rounds-1] < count[rounds-2]:
                                #print("count gets smaller")
                                
                                if rounds % 5 == 0:
                                    if position == 0:
                                        try:
                                            cluster[mentees[j]][0], cluster[mentees[j]][2] = cluster[mentees[j]][2], cluster[mentees[j]][0]
                                            #print("1")
                                        except:
                                            continue
                                    else:
                                        try:
                                            cluster[mentees[i]][0], cluster[mentees[i]][2] = cluster[mentees[i]][2], cluster[mentees[i]][0]
                                            #print("2")          
                                        except:
                                            continue
                                else:
                                    if position == 0:
                                        try:
                                            cluster[mentees[j]][0], cluster[mentees[j]][1] = cluster[mentees[j]][1], cluster[mentees[j]][0]
                                            #print("3")
                                        except:
                                            continue
                                    else:
                                        try:
                                            cluster[mentees[i]][0], cluster[mentees[i]][1] = cluster[mentees[i]][1], cluster[mentees[i]][0]
                                            #print("4")
                                        except: 
                                            continue
                            else:
                                if method == "limited":
                                    if position == 0:
                                        try:
                                            cluster[mentees[j]][1], cluster[mentees[j]][3] = cluster[mentees[j]][3], cluster[mentees[j]][1]
                                            #print("5")
                                        except:
                                            continue
                                    else:
                                        try:    
                                            cluster[mentees[i]][1], cluster[mentees[i]][3] = cluster[mentees[i]][3], cluster[mentees[i]][1] 
                                            #print("6")
                                        except:
                                            continue
                                elif method == "medium":
                                    if position == 0:
                                        try:
                                            cluster[mentees[j]][2], cluster[mentees[j]][4] = cluster[mentees[j]][4], cluster[mentees[j]][2]
                                            #print("7")
                                        except:
                                            continue
                                    else:
                                        try:
                                            cluster[mentees[i]][2], cluster[mentees[i]][4] = cluster[mentees[i]][4], cluster[mentees[i]][2] 
                                            #print("8")
                                        except:
                                            continue                      
                                elif method == "hard": 
                                    indexes = [5,6,7,8,9,10]
                                    random_index = random.choice(indexes)
                                    print("Index:", random_index)
                                    if len(cluster[mentees[i]]) > random_index or len(cluster[mentees[j]]) > random_index:
                                        if position == 0:
                                            try:
                                                cluster[mentees[j]][3], cluster[mentees[j]][random_index] = cluster[mentees[j]][random_index], cluster[mentees[j]][3]
                                                #print("9")
                                            except:
                                                continue
                                        else:
                                            try:
                                                cluster[mentees[i]][3], cluster[mentees[i]][random_index] = cluster[mentees[i]][random_index], cluster[mentees[i]][3]  
                                                #print("11")
                                            except:
                                                continue
                                    else:
                                        if position == 0:
                                            cluster[mentees[j]][0], cluster[mentees[j]][-1] = cluster[mentees[j]][-1], cluster[mentees[j]][0]
                                            #print("13")
                                        else:
                                            cluster[mentees[i]][0], cluster[mentees[i]][-1] = cluster[mentees[i]][-1], cluster[mentees[i]][0] 
                                            #print("14")
                        else:
                            if position == 0:
                                cluster[mentees[j]][0], cluster[mentees[j]][1] = cluster[mentees[j]][1], cluster[mentees[j]][0]
                                #print("15")
                            else:
                                cluster[mentees[i]][0], cluster[mentees[i]][1] = cluster[mentees[i]][1], cluster[mentees[i]][0]   
                                #print("16")             
                    j+=1
            i += 1
            j = 0
        
        test = test_if_unique(cluster)
        if test == "Success":
            return "Success"
        else:
            if rounds == max_rounds-1:
                return "Failed"
            else:
                rounds += 1
                i = 0
                j = 0       
                
def test_if_unique(cluster):
    list_ranks = []
    for mentee in cluster:
        rank1 = cluster[mentee][0]
        list_ranks.append(rank1)
    list_ranks_set = set(list_ranks)
    list_ranks_set = list(list_ranks_set)
    list_ranks_set.sort()
    list_ranks.sort()
    if len(list_ranks) != len(list_ranks_set):
        return ValueError("Failed")
    else:
        return "Success"
    
##### do not touch

In [521]:
test_if_unique(people_cluster_unique)

ValueError('Failed')

In [522]:
# shuffle the assignments until the assignments on the first rank are unique

trys = 1
sorting = sort_unique(people_cluster_unique, method="limited")
trys += 1
while trys < 200:
    if sorting == "Success":
        print("Converged at try:", trys)
        trys = 100
        break
    elif trys < 10:
        sorting = sort_unique(people_cluster_unique, method="limited")
        trys += 1
    else:
        choice_list = ["medium"]*1 + ["hard"]*1 + ["limited"]*1
        random_method = random.choice(choice_list)
        print(random_method)
        sorting = sort_unique(people_cluster_unique, method=random_method)
        trys += 1 
if sorting == "Failed":
    print("Could not converge.")
test_assignment()   

101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
110
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86
86


22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
41
41
41
41
41
41
41
41
41
41
41
41
41
41
41
41
41
41
41
41
41
41
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
43
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
2

22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
34
34
34
34
34
34
34
34
34
3

27
27
27
27
27
27
27
27
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
28
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
33
33
33
33
33
33
33
33
33
33
33
3

20
20
20
20
20
20
20
20
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
2

28
28
28
28
28
28
28
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
43
43
43
43
43
43
4

30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
3

23
Index: 10
23
Index: 10
23
Index: 5
23
Index: 7
23
Index: 9
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
25
Index: 6
25
Index: 5
25
Index: 7
25
Index: 9
25
Index: 7
25
Index: 5
25
Index: 8
25
Index: 8
25
Index: 8
25
Index: 6
25
Index: 9
25
Index: 8
25
Index: 5
25
Index: 7
25
Index: 8
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
22
Index: 9
22
Index: 5
22
Index: 6
22
Index: 9
22
Index: 8
22
Index: 5
22
Index: 5
22
Index: 6
22
Index: 10
22
Index: 8
22
Index: 6
22
Index: 10
22
Index: 10
22
Index: 6
22
Index: 5
22
Index: 5
22
Index: 10
22
Index: 6
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
22
Index: 9
22
Index: 6
22
Index: 8
22
Index: 7
22
Index: 8
22
Index: 6
22
Index: 7
22
Index: 7
22
Index: 7
22
Index: 10
22
Index: 10
22
Index: 10
22
Index: 10
22
Index: 10
22
Index: 10
22
Index: 8
22
Index: 7
22
Index: 7
22
Index: 10
22
Index: 5
22
Index: 9
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
23


19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
29
Index: 5
29
Index: 10
29
Index: 7
29
Index: 5
29
Index: 6
29
Index: 10
29
Index: 9
29
Index: 5
29
Index: 8
29
Index: 6
29
Index: 10
29
Index: 5
29
Index: 9
29
Index: 9
29
Index: 8
29
Index: 8
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
24
Index: 9
24
Index: 7
24
Index: 7
24
Index: 6
24
Index: 9
24
Index: 7
24
Index: 8
24
Index: 9
24
Index: 7
24
Index: 8
24
Index: 6
24
Index: 9
24
Index: 5
24
Index: 5
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
24
Index: 8
24
Index: 10
24
Index: 9
24
Index: 10
24
Index: 5
24
Index: 8
24
Index: 7
24
Index: 9
24
Index: 6
24
Index: 8
24
Index: 5
24
Index: 6
24
Index: 6
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
19
Index: 9
19
Index: 5
19
Index: 10
19
Index: 7
19
Index: 10
19
Index: 8
19
Index: 10
19
Index: 6
19
Index: 7
19
Index: 8
19
Index: 6
19
Index: 10
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
1

1
3
Index: 7
3
Index: 10
3
Index: 8
3
Index: 8
3
Index: 6
2
2
2
2
4
Index: 9
4
Index: 5
2
2
2
3
Index: 6
3
Index: 6
2
2
2
3
Index: 8
3
Index: 6
2
2
2
3
Index: 9
3
Index: 5
2
2
2
Index: 8
2
Index: 5
2
Index: 8
2
Index: 8
2
Index: 7
2
Index: 8
2
Index: 10
2
Index: 10
2
Index: 8
2
Index: 10
2
Index: 5
3
Index: 9
3
Index: 5
2
2
2
2
4
Index: 5
4
Index: 10
4
Index: 7
4
Index: 10
4
Index: 8
4
Index: 5
2
1
1
1
1
4
Index: 8
4
Index: 5
2
2
2
2
2
2
2
2
8
Index: 6
8
Index: 6
2
2
2
3
Index: 10
3
Index: 9
3
Index: 5
3
Index: 7
3
Index: 7
2
1
1
1
1
4
Index: 5
4
Index: 5
2
2
2
3
Index: 10
3
Index: 7
3
Index: 9
medium
4
4
4
3
3
3
3
4
4
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
limited
4
4
2
2
2
2
2
2
6
6
2
2
2
2
2
5
5
2
2
2
3
3
2
2
2
2
2
5
5
2
1
1
1
1
1
1
6
6
2
2
2
2
4
4
2
2
2
2
4
4
2
2
2
2
4
4
2
2
2
2
4
4
2
2
2
2
2
5
5
2
Conve

In [523]:
test_if_unique(people_cluster_unique)

'Success'

# 7. Export Rankings as Excel Sheets

In [524]:
# Create an new Excel file and add a worksheet.
workbook = xlsxwriter.Workbook('Mentee_Mentor_AssignedbyRank_Anonym.xlsx')
worksheet = workbook.add_worksheet()

# format the sheet
worksheet.set_column('A:A', 30)
worksheet.set_column('B:K', 20)
# mentee cell (0,0)
mentee_cell = workbook.add_format({'bold': True})
mentee_cell.set_right()
mentee_cell.set_bottom()
# vertical
vertical = workbook.add_format({'bold': True})
vertical.set_right()
# horzontal
horizontal = workbook.add_format({'bold': True})
horizontal.set_bottom()

# Write "Mentee" cell
worksheet.write(0, 0, "Mentee", mentee_cell)


# Write mentee names
i = 1
for mentee in people_cluster_unique:
    worksheet.write(i, 0, str(mentee), vertical)
    i+=1

# Write ranks
i = 1
for j in range(0,10):
    worksheet.write(0, i, str("Rank " + str(i)), horizontal)
    i+=1

# Write mentor names
row = 1
for mentee in people_cluster_unique:
    mentors = people_cluster_unique[mentee]
    col = 1
    for mentor in mentors:
        worksheet.write(row, col, mentor)
        col+=1
    row +=1
    
workbook.close()

# 8. Test results

In [529]:
# function that tests whether a mentee has a mentor on rank 1 that is his most similar mentor
def rank_test():
    error_count = 0
    not_nr1_count = 0
    for mentee in people_similarity.columns:
        try:
            mentors_similar = list(people_similarity[mentee].nsmallest().index)
            mentors_assigned = people_cluster[mentee]
            for top_mentors in mentors_similar:
                if top_mentors not in mentors_similar:
                    error_count += 1
            if error_count != 0:
                raise ValueError("Error! Mentee", mentee, "has not all of its top 5 mentors in the final ranking.")
            if mentors_similar[0] != mentors_assigned[0]:
                not_nr1_count += 1
        except:
            continue
    print("Test passed.")
    print(not_nr1_count, "/", len(people_similarity.columns), "do not have mentor nr1 on rank1.")

In [530]:
rank_test()

Test passed.
51 / 79 do not have mentor nr1 on rank1.
